In [1]:
%load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths


import sys, os
sys.path.append(os.path.abspath('../..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.5 



<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.7.6, build 905165378
Copyright (C) 2014-2023 Monash University, NICTA, Data61


In [2]:
exchange = 'kucoin'
pair = 'BTC_USDT'
timeframe = '4h'

odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

# odf['idate'] = odf.dtime.dt.strftime('%Y%m%d')
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)

odf.shape, odf.head(5)

((7701, 6),
                          timestamp    open    high     low   close     volume
 dtime                                                                        
 2020-01-01 00:00:00  1577836800000  7196.2  7243.2  7176.1  7225.4  62.743859
 2020-01-01 04:00:00  1577851200000  7225.4  7233.8  7203.1  7209.5  36.248625
 2020-01-01 08:00:00  1577865600000  7209.4  7232.0  7183.0  7195.6  52.545595
 2020-01-01 12:00:00  1577880000000  7195.7  7251.8  7195.6  7231.4  69.704852
 2020-01-01 16:00:00  1577894400000  7231.5  7248.0  7217.1  7229.0  60.889127)

In [4]:
# odf.close.plot()

In [5]:
#
h_columns = ['open', 'high', 'low', 'close', 'volume']
resampling_period = '1D'
hdf = odf[h_columns].resample(resampling_period).agg(
    OrderedDict([
        ('open', 'first'),
        ('high', 'max'),
        ('low', 'min'),
        ('close', 'last'),
        ('volume', 'sum'),
    ])
)

gbsz = odf[[]].groupby(pd.Grouper(freq=resampling_period)).size()
odf['h_dtime'] = sorted([c for b in [[t]*s for t,s in gbsz.reset_index().values] for c in b])

odf = odf.join(hdf[['open','high','low','close','volume']].set_axis(['h_open','h_high','h_low','h_close','h_volume'], axis=1), on='h_dtime')
odf.head(10)

,timestamp,open,high,low,close,volume,h_dtime,h_open,h_high,h_low,h_close,h_volume
dtime,,,,,,,,,,,,
2020-01-01 00:00:00,1577836800000,7196.2,7243.2,7176.1,7225.4,62.743859,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 04:00:00,1577851200000,7225.4,7233.8,7203.1,7209.5,36.248625,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 08:00:00,1577865600000,7209.4,7232.0,7183.0,7195.6,52.545595,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 12:00:00,1577880000000,7195.7,7251.8,7195.6,7231.4,69.704852,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 16:00:00,1577894400000,7231.5,7248.0,7217.1,7229.0,60.889127,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 20:00:00,1577908800000,7229.0,7240.0,7180.0,7200.2,88.462003,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-02 00:00:00,1577923200000,7200.3,7210.6,7119.5,7125.6,106.717570,2020-01-02,7200.3,7210.6,6933.3,6964.8,834.203799
2020-01-02 04:00:00,1577937600000,7126.2,7158.3,7104.1,7108.8,134.437394,2020-01-02,7200.3,7210.6,6933.3,6964.8,834.203799
2020-01-02 08:00:00,1577952000000,7108.9,7178.0,7108.2,7144.9,92.622499,2020-01-02,7200.3,7210.6,6933.3,6964.8,834.203799


In [6]:
### get dataframe of original and resampled time
smpl_df = pd.DataFrame(np.array([[k,v] for k, idxs in odf[[]].resample(resampling_period).indices.items() for v in odf.iloc[idxs].index.values])
).set_axis(['h_dtime', 'time'], axis=1).sort_values(by='time')
smpl_df


,h_dtime,time
0,2020-01-01,2020-01-01 00:00:00
1,2020-01-01,2020-01-01 04:00:00
2,2020-01-01,2020-01-01 08:00:00
3,2020-01-01,2020-01-01 12:00:00
4,2020-01-01,2020-01-01 16:00:00
...,...,...
7696,2023-07-06,2023-07-06 16:00:00
7697,2023-07-06,2023-07-06 20:00:00
7698,2023-07-07,2023-07-07 00:00:00
7699,2023-07-07,2023-07-07 04:00:00


In [6]:
# hupeaks 
# np.array([[hdf.loc[iidx, 'close']] + df.index[[0,-1]].tolist() 
#                            for iidx in hupeaks
#                            for idx in [hdf[[]].iloc[iidx].index]
#                            for df in [wdf[wdf.h_dtime == iidx]]
#                         ])

# wdf

In [7]:
wdf,hdf, df_hmap = None,None,None
ihupeaks, hupeak_props = None, None
ihdpeaks = None
def get_indicators(**kwargs):
    global df_hmap
    global ihupeaks, ihdpeaks #, hupeak_props, hupeaks_idxs, last_upeak_iidxs, last_upeaks

    wdf, pkwargs = kwargs['wdf'], kwargs['pkwargs']
    # print('pkwargs', pkwargs)
    # wdf = pkwargs['wdf']
    inow, pkdist, pct_hpeak = pkwargs['inow'], pkwargs['pkdist'],pkwargs['pct_hpeak']

    ### resampling to higher timeframe for peaks
    h_columns = ['open', 'high', 'low', 'close', 'volume']
    hsample = pkwargs['hsample']
    hdf = wdf[h_columns].resample(hsample).agg(
        OrderedDict([
            ('open', 'first'),
            ('high', 'max'),
            ('low', 'min'),
            ('close', 'last'),
            ('volume', 'sum'),
        ])
    )
    df_hmap = pd.DataFrame(np.array([[k,v] for k, idxs in odf[[]].resample(resampling_period).indices.items() for v in odf.iloc[idxs].index.values])
).set_axis(['h_dtime', 'time'], axis=1).sort_values(by='time')
    # whdf = hdf.loc[wdf.index.date[0]:wdf.index.date[-1]]
    # ixnow = wdf.index[]
    # hix_now = wdf.iloc[inow].h_dtime.

    ihupeaks, hupeak_props = find_peaks(hdf.loc[:wdf.iloc[inow].h_dtime].iloc[:-1].close, prominence=1, distance=pkdist)
    ihdpeaks, hdpeak_props = find_peaks(-hdf.loc[:wdf.iloc[inow].h_dtime].iloc[:-1].close, prominence=1, distance=pkdist)
    
    # hdpeaks, hdpeak_props = find_peaks(-hdf.close, prominence=1, distance=pkdist)
    # print('upeaks: ', upeaks)
    # print('wdf:', wdf)
    # results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
    # results_full = peak_widths(wdf.close, peaks, rel_height=1)
    
    # hupeaks_idxs = hdf.index[hupeaks]
    # hdpeaks_idxs = hdf.index[hdpeaks]


    # last_upeak_iidxs = ([ p[-1] if p.shape[0] > 0 else pd.NA for p in [hupeaks[hupeaks < i] for i in range(hdf.shape[0])] ])
    # gwdf['upeak_close'] = 
    # last_upeak_prices = [ hdf.iloc[i]['close'] if not pd.isna(i) else pd.NA for i in last_upeak_iidxs]

    return {'wdf': wdf, 'hdf': hdf, 'df_hmap': df_hmap, 'ihupeaks': ihupeaks, 'ihdpeaks': ihdpeaks} 
# (hupeaks, hupeaks_idxs, last_upeak_iidxs, last_upeak_prices), hdpeaks_idxs, wdf, hdf

signals = None
def get_signals(indicators, pkwargs):
    global wdf, hdf, signals
    wdf = indicators['wdf']
    hdf = indicators['hdf']
    ihupeaks = indicators['ihupeaks']
    ihdpeaks = indicators['ihdpeaks']
    # (hupeaks, hupeaks_idxs, last_upeak_iidxs, last_upeak_prices), dpeaks_idx, whdf = indicators
    pct_hpeak = pkwargs['pct_hpeak']
    
    # dist_last_upeaks = ((last_upeak_prices / wdf.close) - 1).abs()
    # signals = (dist_last_upeaks >= pct_hpeak[0]) & (dist_last_upeaks <= pct_hpeak[1])

    # print('pct_hpeak', pct_hpeak)
    # upeaks, upeak_props = find_peaks(wdf.close, distance=pkdist)
    # dpeaks, dpeak_props = find_peaks(-wdf.close, distance=pkdist)

    # # results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
    # # results_full = peak_widths(wdf.close, peaks, rel_height=1)
    
    # upeaks_idxs = wdf.index[upeaks]
    # dpeaks_idxs = wdf.index[dpeaks]

    # close_peaks = wdf.loc[peaks_idx].close
    return signals #upeaks_idxs,dpeaks_idxs

In [101]:
# ihupeaks
list(range(1,5+1))

[1, 2, 3, 4, 5]

In [106]:
pjn=7
[
    []
    + [hdf.loc[hidx, 'close']]
    + df.index[[0,-1]].tolist()
    for ihu in ihupeaks
    for [hidx, ehidx] in [hdf.index.values[[ihu, min(hdf.shape[0]-1, ihu+pjn)]]]
    for df in [ wdf[(wdf.h_dtime >= hidx) & (wdf.h_dtime <= ehidx)] ]
]

[[7376.5, Timestamp('2020-04-07 00:00:00'), Timestamp('2020-04-09 20:00:00')],
 [7359.7, Timestamp('2020-04-09 00:00:00'), Timestamp('2020-04-11 20:00:00')],
 [7284.9, Timestamp('2020-04-10 00:00:00'), Timestamp('2020-04-12 20:00:00')],
 [6877.4, Timestamp('2020-04-11 00:00:00'), Timestamp('2020-04-13 20:00:00')],
 [7015.7, Timestamp('2020-04-13 00:00:00'), Timestamp('2020-04-14 20:00:00')],
 [6906.2, Timestamp('2020-04-15 00:00:00'), Timestamp('2020-04-16 20:00:00')]]

In [107]:

### Ranges
# nest_asyncio.apply()

strategy = 'swings'
strategy_params_json = f'./par-{strategy}.json'

strategy_params = {
    "pkdist": {"wdg": IntSlider(value=7,min=1,max=100,step=1,description='PKDist',continuous_update=False)},
    "hsample": {"wdg": Text(value='1D',description='hsample',continuous_update=False)},
    "pjn": {"wdg": IntSlider(value=7,min=0,max=100,step=1,description='pjn',continuous_update=False)},
    # "pkprom": {"wdg": FloatRangeSlider(value=(0.05,1),min=0,max=1,step=0.005,description='PKProm',continuous_update=False)},
    "pct_hpeak": {"wdg": FloatRangeSlider(value=(0,0.1),min=0,max=0.2,step=0.001,description='dhp',continuous_update=False, readout_format='.3f')},
}

mpl.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 300, 'lines.markersize': 4})
plt.rcParams['lines.linewidth'] = 0.4

def draw_func(ax_dict, indicators, signals, pkwargs):
    axa = ax_dict['A']

    wdf = indicators['wdf']
    hdf = indicators['hdf']
    ihupeaks = indicators['ihupeaks']
    ihdpeaks = indicators['ihdpeaks']
    df_hmap = indicators['df_hmap']
    # (hupeaks, hupeaks_idxs, last_upeak_iidxs, last_upeak_prices), dpeaks_idx, whdf = indicators
    # print('indicators: ', indicators)
    # print(wdf.close[upeaks_idx])
    
    # axa.scatter(upeaks_idxs, whdf.close[upeaks_idxs], s=50, c='r', alpha=0.2)
    # axa.scatter(dpeaks_idx, whdf.close[dpeaks_idx], s=50, c='g', alpha=0.2)
    # hupeaks_idxs = hdf.index[hupeaks]
    pjn = pkwargs['pjn']
    upeak_spans = np.array([
        []
        + [hdf.loc[hidx, 'close']]
        + df.index[[0,-1]].tolist()
        for ihu in ihupeaks
        for [hidx, ehidx] in [hdf.index.values[[ihu, min(hdf.shape[0]-1, ihu+pjn)]]]
        for df in [ wdf[(wdf.h_dtime >= hidx) & (wdf.h_dtime <= ehidx)] ]
    ])
    upeak_spansT = upeak_spans.T
    dpeak_spans = np.array([
        []
        + [hdf.loc[hidx, 'close']]
        + df.index[[0,-1]].tolist()
        for ihu in ihdpeaks
        for [hidx, ehidx] in [hdf.index.values[[ihu, min(hdf.shape[0]-1, ihu+pjn)]]]
        for df in [ wdf[(wdf.h_dtime >= hidx) & (wdf.h_dtime <= ehidx)] ]
    ])
    dpeak_spansT = dpeak_spans.T
    # print('upeak_spansT', upeak_spansT)

    hdf.close.plot(ax=axa, lw=2, c='m', alpha=0.3)
    # n_fu = 6
    # for i in range(len(upeak_spansT)): 
    if len(upeak_spans) > 0: 
        axa.hlines(upeak_spansT[0], xmin=upeak_spansT[1], xmax=upeak_spansT[2], colors='r', linewidths=1.5, alpha=0.3)
        # axa.hlines(upeak_spansT[0], xmin=upeak_spansT[3], xmax=upeak_spansT[4], colors='o', linewidths=1.5, alpha=0.3)
    if len(dpeak_spans) > 0: axa.hlines(dpeak_spansT[0], xmin=dpeak_spansT[1], xmax=dpeak_spansT[2], colors='g', linewidths=1.5, alpha=0.3)
    
    # xmin,xmax = np.array(upeaks_idxs, wdf.index.values)
    # axa.hlines(wdf.close, [ for i in ], wdf.index)
    # for idx in wdf.index[signals]: 
    #     for ax in [axa]: ax.axvline(idx)
    # axa.plot(dpeaks_idx, wdf.close[upeaks_idx], "^")
    

fu_printer(odf, strategy, strategy_params = strategy_params, draw_func = draw_func, indicator_func=get_indicators, signal_func=get_signals)

#     ax1.plot(peaks_idx, close_peaks, "^", c='m', markersize=2, alpha=0.5)
#     ax1.vlines(x=peaks_idx, ymin=close_peaks - peak_props["prominences"],
#            ymax = close_peaks, color = "C1", lw=0.2)
#     ax1.hlines(results_full[1:][0], xmin=wdf.index[results_full[1:][1].astype(int)], xmax=wdf.index[results_full[1:][2].astype(int)], color="C1", lw=0.3)
#     ax1.hlines(results_half[1:][0], xmin=wdf.index[results_half[1:][1].astype(int)], xmax=wdf.index[results_half[1:][2].astype(int)], color="C2", lw=0.3)
#     # ax1.hlines(y=(close_peaks - peak_props["prominences"]), xmin=wdf.index[peak_props["left_bases"]],
#     #        xmax=wdf.index[peak_props["right_bases"]], color = "C1")




Output()

In [59]:

# dist_last_upeaks = ((last_upeak_prices / wdf.close) - 1).abs()
# signals = (dist_last_upeaks >= pct_hpeak[0]) & (dist_last_upeaks <= pct_hpeak[1])

# print('pct_hpeak', pct_hpeak)
# upeaks, upeak_props = find_peaks(wdf.close, distance=pkdist)
# dpeaks, dpeak_props = find_peaks(-wdf.close, distance=pkdist)

# # results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
# # results_full = peak_widths(wdf.close, peaks, rel_height=1)

# df_hmap = pd.DataFrame(
# np.array([[k,v] for k, idxs in odf[[]].resample(resampling_period).indices.items() for v in odf.iloc[idxs].index.values])
pjn = 7
# np.array([
#     [hdf.loc[hidx, 'close']] + df.index[[0,-1]].tolist()                        
#     for ihu in ihupeaks
#     for [hidx, ehidx] in [hdf.index.values[[ihu, min(hdf.shape[0]-1, ihu+pjn)]]]
#     for df in [ wdf[(wdf.h_dtime >= hidx) & (wdf.h_dtime <= ehidx) ] ]
# ])

df_hclose = pd.DataFrame(
    np.array([
        [hidx, widx, hdf.loc[hidx, 'close']]                      
        for ihu in ihupeaks
        for [hidx, hidx1, ehidx] in [hdf.index.values[[ihu, ihu+1, min(hdf.shape[0]-1, ihu+pjn)]]]
        for widx in wdf[(wdf.h_dtime > hidx) & (wdf.h_dtime <= ehidx) ].index
    ])
).set_axis(['pk_h_dtime', 'dtime', 'pk_h_close'], axis=1)#.set_index('date').sort_index()


gb = df_hclose.groupby('dtime')

tdf = wdf.join(gb['pk_h_close'].apply(lambda x: x.tolist())
).join(gb['pk_h_dtime'].apply(lambda x: x.tolist()))#.to_csv('test.csv')

tdf.to_csv('test.csv')
# df_hclose.assign(pk_h_close = gb['pk_h_close'].apply(lambda x: x.tolist())
#                  ).assign(pk_h_dtime=gb['pk_h_dtime'].apply(lambda x: x.tolist())).to_csv('test.csv')
# gb.apply(lambda x: x.pk_h_dtime.tolist())
# df_hclose.assign(gb.apply(pk_h_close=lambda x: x.pk_h_close.tolist())).to_csv('test.csv')
# df_hclose.join(wdf,how='left')
# zdf = wdf.join(df_hclose[['pk_h_dtime', 'pk_h_close']])
# zdf.to_csv('test.csv')
# zdf.shape
#.dropna()#.drop_duplicates()


In [17]:
wdf[[]].resample(resampling_period).indices.items()

dict_items([(Timestamp('2020-01-01 00:00:00'), [0, 1, 2, 3, 4, 5]), (Timestamp('2020-01-02 00:00:00'), [6, 7, 8, 9, 10, 11]), (Timestamp('2020-01-03 00:00:00'), [12, 13, 14, 15, 16, 17]), (Timestamp('2020-01-04 00:00:00'), [18, 19, 20, 21, 22, 23]), (Timestamp('2020-01-05 00:00:00'), [24, 25, 26, 27, 28, 29]), (Timestamp('2020-01-06 00:00:00'), [30, 31, 32, 33, 34, 35]), (Timestamp('2020-01-07 00:00:00'), [36, 37, 38, 39, 40, 41]), (Timestamp('2020-01-08 00:00:00'), [42, 43, 44, 45, 46, 47]), (Timestamp('2020-01-09 00:00:00'), [48, 49, 50, 51, 52, 53]), (Timestamp('2020-01-10 00:00:00'), [54, 55, 56, 57, 58, 59]), (Timestamp('2020-01-11 00:00:00'), [60, 61, 62, 63, 64, 65]), (Timestamp('2020-01-12 00:00:00'), [66, 67, 68, 69, 70, 71]), (Timestamp('2020-01-13 00:00:00'), [72, 73, 74, 75, 76, 77]), (Timestamp('2020-01-14 00:00:00'), [78, 79, 80, 81, 82, 83]), (Timestamp('2020-01-15 00:00:00'), [84, 85, 86, 87, 88, 89]), (Timestamp('2020-01-16 00:00:00'), [90, 91, 92, 93, 94, 95]), (Times

In [16]:
### get dataframe of original and resampled time
pd.DataFrame(np.array([[k,v] for k, idxs in wdf[[]].resample(resampling_period).indices.items() for v in wdf.iloc[idxs].index.values])
).set_axis(['h_dtime', 'time'], axis=1).sort_values(by='time')


,h_dtime,time
0,2020-01-01,2020-01-01 00:00:00
1,2020-01-01,2020-01-01 04:00:00
2,2020-01-01,2020-01-01 08:00:00
3,2020-01-01,2020-01-01 12:00:00
4,2020-01-01,2020-01-01 16:00:00
...,...,...
251,2020-02-11,2020-02-11 20:00:00
252,2020-02-12,2020-02-12 00:00:00
253,2020-02-12,2020-02-12 04:00:00
254,2020-02-12,2020-02-12 08:00:00


In [18]:
wdf

,timestamp,open,high,low,close,volume,h_dtime,h_open,h_high,h_low,h_close,h_volume
dtime,,,,,,,,,,,,
2020-01-01 00:00:00,1577836800000,7196.2,7243.2,7176.1,7225.4,62.743859,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 04:00:00,1577851200000,7225.4,7233.8,7203.1,7209.5,36.248625,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 08:00:00,1577865600000,7209.4,7232.0,7183.0,7195.6,52.545595,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 12:00:00,1577880000000,7195.7,7251.8,7195.6,7231.4,69.704852,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
2020-01-01 16:00:00,1577894400000,7231.5,7248.0,7217.1,7229.0,60.889127,2020-01-01,7196.2,7251.8,7176.1,7200.2,370.594061
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-11 20:00:00,1581451200000,10243.4,10253.2,10173.1,10222.5,166.004733,2020-02-11,9853.3,10317.0,9715.6,10222.5,1536.236839
2020-02-12 00:00:00,1581465600000,10222.4,10321.2,10222.4,10261.7,223.241861,2020-02-12,10222.4,10445.0,10091.6,10330.0,1459.221128
2020-02-12 04:00:00,1581480000000,10261.6,10445.0,10257.8,10370.0,246.472409,2020-02-12,10222.4,10445.0,10091.6,10330.0,1459.221128


In [76]:

peaks, peak_props = find_peaks(wdf.close, prominence=1, distance=10)
# wdf.iloc[peaks].close

results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
# np.shape(results_half[1:])
results_half[1:][1].astype(int)
# fig, (ax1,ax2, ax3) = plt.subplots(3, 1, height_ratios=[3,1,1], sharex=True);

# plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
# plt.plot(peaks, wdf.iloc[peaks].close, "x")

array([  1,   9,  24,  35,  49,  61,  81,  91, 101, 112, 127, 142, 159,
       172, 189, 202, 218, 229, 245, 240, 266, 279, 302, 316, 320, 338,
       347, 358, 366, 383, 381, 406, 418, 428, 439, 450, 463, 480, 501])

In [56]:
tuple([1,2])

(1, 2)